# DEPENDÊNCIAS

In [ ]:
! pip install spacy
! pip install nltk
! pip install pandas
! pip install pdfplumber
! pip install chromadb
! pip install sentence-transformers
! pip install openpyxl
! pip install python-docx
! python -m spacy download pt_core_news_sm
! pip install langchain-text-splitters

# IMPORTS

In [2]:
import pdfplumber
import re
import unicodedata
import spacy
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb

In [3]:
pdf_path = "./documents/sample_vangogh.pdf"

# Carrega PDF

In [5]:
# Utiliza a bibliotéca pdfplumber para extrair o texto do PDF, o resultado é guardado na variável raw_text
def load_pdf(filepath):
    with pdfplumber.open(filepath) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text



raw_text = load_pdf(pdf_path)
print("Prova de Conceito:\n\n" + raw_text[:500])

Prova de Conceito:

Prólogo
Um coração fanático
Theo imaginou o pior. A mensagem dizia apenas que Vincent tinha “se ferido”.
Enquanto corria até a estação para pegar o próximo trem até Auvers, seus pensa-
mentos vagavam de trás para diante. Da última vez que havia recebido uma mensa-
gem parecida, era um telegrama de Paul Gauguin informando que Vincent estava
“gravemente enfermo”. Theo chegara à cidade meridional de Arles e encontrara o
irmão na enfermaria de um hospital, com a cabeça enfaixada e perdido em desvari


# PLN

## Lowercasing

In [8]:
# Transforma todo o texto em letras minúsculas
text_lower = raw_text.lower()




print("Texto em lowercase (prévia):\n\n", text_lower[:800])

Texto em lowercase (prévia):

 prólogo
um coração fanático
theo imaginou o pior. a mensagem dizia apenas que vincent tinha “se ferido”.
enquanto corria até a estação para pegar o próximo trem até auvers, seus pensa-
mentos vagavam de trás para diante. da última vez que havia recebido uma mensa-
gem parecida, era um telegrama de paul gauguin informando que vincent estava
“gravemente enfermo”. theo chegara à cidade meridional de arles e encontrara o
irmão na enfermaria de um hospital, com a cabeça enfaixada e perdido em desvarios.
dessa vez, o que encontraria ao final da viagem?
em momentos assim — e eram muitos —, theo retornava às lembranças do
vincent que tinha conhecido no passado: um irmão mais velho ardoroso e irrequie-
to, mas também cheio de brincadeiras animadas, uma enorme afinidade e uma infi-
nita capacidade d


## Remover caracteres especiais

In [9]:
def remove_special_chars(text):
    # Normaliza acentos (NFKD)
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ASCII", "ignore").decode("utf-8")

    # Trata hifens seguidos por quebra de linha como quebra de palavra
    text = re.sub(r"-\n", "", text);
    # Remove caracteres que não sejam letras, numeros ou espaços
    text = re.sub(r"[^a-z0-9.,\s]", " ", text)

    # Remove múltiplos espaços
    text = re.sub(r"\s+", " ", text)

    return text.strip()




text_clean = remove_special_chars(text_lower)

print("Texto sem caracteres especiais:\n\n", text_clean[:800])


Texto sem caracteres especiais:

 prologo um coracao fanatico theo imaginou o pior. a mensagem dizia apenas que vincent tinha se ferido. enquanto corria ate a estacao para pegar o proximo trem ate auvers, seus pensamentos vagavam de tras para diante. da ultima vez que havia recebido uma mensagem parecida, era um telegrama de paul gauguin informando que vincent estava gravemente enfermo. theo chegara a cidade meridional de arles e encontrara o irmao na enfermaria de um hospital, com a cabeca enfaixada e perdido em desvarios. dessa vez, o que encontraria ao final da viagem em momentos assim e eram muitos , theo retornava as lembrancas do vincent que tinha conhecido no passado um irmao mais velho ardoroso e irrequieto, mas tambem cheio de brincadeiras animadas, uma enorme afinidade e uma infinita capacidade de admiracao. em s


# Tokenização

In [11]:
# A tokenização sera feita com spaCy pois funciona melhor com português, em comparação com nltk.
# Para isso, basta carregar modelo de português

nlp = spacy.load("pt_core_news_sm")
token = nlp(text_clean)

print("Tokenização (prévia):\n\n", [t.text for t in token[:50]])

Tokenização (prévia):

 ['prologo', 'um', 'coracao', 'fanatico', 'theo', 'imaginou', 'o', 'pior', '.', 'a', 'mensagem', 'dizia', 'apenas', 'que', 'vincent', 'tinha', 'se', 'ferido', '.', 'enquanto', 'corria', 'ate', 'a', 'estacao', 'para', 'pegar', 'o', 'proximo', 'trem', 'ate', 'auvers', ',', 'seus', 'pensamentos', 'vagavam', 'de', 'tras', 'para', 'diante', '.', 'da', 'ultima', 'vez', 'que', 'havia', 'recebido', 'uma', 'mensagem', 'parecida', ',']


# Chunking


In [13]:
# Dividir o texto em chunks utilizando RecursiveCharacterTextSplitter do LangChain
# corta o texto, tentando preservar a integridade semântica dos pedaços, utilizando uma lista de separadores.
# os separadores usados nesse caso foram: dupla quebra de linha (para titulos e paragrafos), quebra de linha padrão, ponto e virgula para frases, e espaços para palavras.
# isso visa ir do mais amplo para o mais específico.
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=15,
    separators=["\n\n", "\n", ".", ",", " ", ""]
)




chunks_recursive = recursive_splitter.split_text(token.text)

print(f"Total de chunks: {len(chunks_recursive)}\n\n", chunks_recursive)

Total de chunks: 299

 ['prologo um coracao fanatico theo imaginou o pior. a mensagem dizia apenas que vincent tinha se ferido', '. enquanto corria ate a estacao para pegar o proximo trem ate auvers, seus pensamentos vagavam de tras para diante', '. da ultima vez que havia recebido uma mensagem parecida, era um telegrama de paul gauguin informando que vincent estava gravemente enfermo', '. theo chegara a cidade meridional de arles e encontrara o irmao na enfermaria de um hospital, com a cabeca enfaixada e perdido em desvarios', '. dessa vez, o que encontraria ao final da viagem em momentos assim e eram muitos', ', theo retornava as lembrancas do vincent que tinha conhecido no passado um irmao mais velho ardoroso e irrequieto', ', mas tambem cheio de brincadeiras animadas, uma enorme afinidade e uma infinita capacidade de admiracao', '. em seus passeios de infancia nos campos e matas ao redor da cidade holandesa de zundert, onde haviam nascido', ', vincent o apresentava as belezas e mis

# Embedding

In [14]:
# Transforma os chunks em embeddings, para que possam ser inseridos no banco vetorial.
# Utilizar modelo pré-treinado de Sentence Transformers para gerar embeddings
# Escolhi o modelo "rufimelo/bert-base-portuguese-cased-sts" pois foi treinado para tarefas de similaridade semântica em português.
# Nos testes, ele teve uma performance melhor do que outros modelos multilingues.
model_name = "rufimelo/bert-base-portuguese-cased-sts"
model = SentenceTransformer(model_name)

def embed_hf(chunks):
    vectors = model.encode(chunks, show_progress_bar=True)

    print(f"\nDimensão: {vectors.shape}\n\nResultado: \n", vectors[:5])

    return vectors

# Testes
embedding = embed_hf(chunks_recursive)


Invalid model-index. Not loading eval results into CardData.
Batches: 100%|██████████| 10/10 [00:16<00:00,  1.63s/it]


Dimensão: (299, 1024)

Resultado: 
 [[ 0.7861579  -0.80223936  0.25168613 ...  0.7063509   0.30981445
   0.9744765 ]
 [-0.46321002 -0.6288214  -0.29497406 ...  0.3188033   0.14358622
   0.4461091 ]
 [ 0.53917456 -0.07000658  0.7654278  ...  0.94934046  0.04997751
   0.48626915]
 [-0.3553681  -0.29860523  1.2897855  ...  0.2666336   0.41614252
  -0.41730386]
 [ 0.08949019 -0.48762417  0.78600276 ...  0.48531693 -1.5037134
   0.36837655]]


# Banco Vetorial

## Criação/Inserção ChromaDb

In [15]:
# Inicializa o cliente ChromaDB e cria uma coleção para armazenar os embeddings
client = chromadb.Client()
collection_name = "exemplo_colecao_sem_lemma"

# resetando coleção caso exista (para testes)
if collection_name in [col.name for col in client.list_collections()]:
    client.delete_collection(collection_name)

collection = client.get_or_create_collection(
    name=collection_name,
    embedding_function=None,
)



print(f"{collection.count()} documentos na coleção")

0 documentos na coleção


In [16]:
# Insere os embeddings na coleção do ChromaDB, criando IDs únicos para cada chunk
ids = [f"chunk_{i}" for i in range(len(chunks_recursive))]

collection.add(
    ids=ids,
    documents=chunks_recursive,
    embeddings=embedding
)




print("Embeddings inseridos com sucesso!")


Embeddings inseridos com sucesso!


## Busca Semântica

In [17]:
# Realiza uma consulta de similaridade no banco vetorial utilizando uma pergunta de exemplo
# Pega uma pergunta, e converte em embeddingm, no mesmo modelo usado para criar os embeddings dos chunks.
question = "Qual o nome de Van Gogh?"
question_embedding = model.encode([question])

# Realiza a consulta no banco vetorial utilizando o embedding da pergunta, e pega os 3 resultados mais similares.
resultados = collection.query(
    query_embeddings=question_embedding.tolist(),
    n_results=3
)

print(f"\nPergunta: {question}")

for doc, score in zip(resultados['documents'][0], resultados['distances'][0]):
    print(f"(Score: {score}) - Documento: {doc}")

# os scores mostram a similaridade entre a pergunta e o chunk retornado. Scores mais baixos indicam maior similaridade.


Pergunta: Qual o nome de Van Gogh?
(Score: 296.7704162597656) - Documento: , chamado van gogh
(Score: 355.330810546875) - Documento: . ninguem acreditava na importancia da biografia com fervor maior do que vincent van gogh
(Score: 381.7509765625) - Documento: , e sua doenca so foi reconhecida muito mais tarde pelo sobrinho, o pintor vincent van gogh
